## Optimization using gradient descent

This notebook allows one to do conduct uni- and multivariate linear fits using the gradient descent method. 


In [1]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

import sys
sys.path.append('/Users/ondrea/MLandstats/OStats/')
from ostats import ML
from ostats import dplot as dp
#from mpltools import color as colors

In [2]:
data_range = np.random.RandomState(1) #make up some fake data for testing
fakex = 10 * data_range.rand(50)
fakey = 3 * fakex - 5 + data_range.randn(50)


!rm -rf ./data/housing*
!wget -P ./data https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
    
#get some data for testing
!wget -P ./data https://datahub.io/core/global-temp/r/annual.csv

--2021-05-26 12:13:05--  https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘./data/housing.csv’

housing.csv         100%[===================>]   1.36M  7.10MB/s    in 0.2s    

2021-05-26 12:13:06 (7.10 MB/s) - ‘./data/housing.csv’ saved [1423529/1423529]

--2021-05-26 12:13:06--  https://datahub.io/core/global-temp/r/annual.csv
Resolving datahub.io (datahub.io)... 104.21.40.221, 172.67.157.38
Connecting to datahub.io (datahub.io)|104.21.40.221|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/global-temp/annual_csv/data/a26b154688b061cdd04f1df36e4408be/annual_csv.csv [fol

In [3]:
year, tempa = np.loadtxt('./data/annual.csv', skiprows=1, delimiter=",", usecols=(1,2), unpack=True)
year = np.array([int(y) for y in year])/np.max(year); tempa=np.array(tempa)+1

In [4]:
year = 1 + (year - np.mean(year))/(np.max(year)-np.min(year))

In [5]:
#year

In [6]:
housing_data = pd.read_csv('./data/housing.csv')

In [7]:
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [8]:
housing_data = housing_data[housing_data.median_house_value != 500001] #a bunch of values are set to this
housing_data = housing_data.sample(n=30) #smaller sample
housing_data.median_house_value = housing_data.median_house_value/100000
housing_data.total_bedrooms = housing_data.total_bedrooms/1000

In [9]:
housing_data.median_house_value.max()

4.661

In [10]:
#housing_data

In [11]:
x_house = housing_data['median_house_value'].to_numpy()
y_house = housing_data['total_bedrooms'].to_numpy()

In [12]:
def plotfit(x, beta):
    funct = beta[-1][0]+x*beta[-1][1]
    return(funct)

def plotfit2(x, beta):
    functs=[]
    points = np.linspace(0,len(beta),10)
    points = [int(x) for x in points]
    for i in range(len(points)-1):
        functs.append(beta[points[i]][0]+x*beta[points[i]][1])
    return(functs)

### Fake data

In [13]:
ifig=1;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta_fake, J_fake = ML.GradDes_Regression(x=fakex, y=fakey, gamma=0.01, l=1)
plt.scatter(fakex,fakey, c='darkred',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(fakex,plotfit2(fakex,beta=theta_fake)[i-1], color=dp.ColorGradient(stop=(11, 0, 59), n=nlines)[i],lw=1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

715


/Users/ondrea/MLandstats/OStats/ostats/ML.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] - store_theta[-1][0])/store_theta[-1][0] < epsilon)


### Housing data

In [24]:
ifig=2;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta_house, J_house = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.001, epsilon = 0.001, l=1)
theta_house_alt, J_house_alt = ML.GradDes_Regression(x=x_house, y=y_house, gamma=0.01,epsilon = 0.001, l=1)
plt.scatter(x_house, y=y_house, c='g',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(x_house,plotfit2(x_house,beta=theta_house)[i-1],\
             color=dp.ColorGradient(stop=(0, 200, 90), n=nlines)[i],lw=1)
    plt.plot(x_house,plotfit2(x_house,beta=theta_house_1)[i-1],\
             color=dp.ColorGradient(stop=(140, 20, 90), n=nlines)[i],lw=1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

524
666


/Users/ondrea/MLandstats/OStats/ostats/ML.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  stop_condition = (np.abs(theta[1] - store_theta[-1][1])/store_theta[-1][1] < epsilon) and (np.abs(theta[0] - store_theta[-1][0])/store_theta[-1][0] < epsilon)


In [15]:
### Climate data

In [16]:
ifig=3;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta_climate, J_climate = ML.GradDes_Regression(x=year, y=tempa, gamma=0.01, l=1, theta_init = np.array([0.5,0.5]))
plt.scatter(year, y=tempa, c='orange',  s=15)
nlines = 10
for i in range(nlines):
    plt.plot(year,plotfit2(year,beta=theta_climate)[i-1],\
             color=dp.ColorGradient(stop=(31, 80, 40), n=nlines)[i],lw=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

6703


## Now take a look at convergence by plotting $J(\theta)$ vs $(\theta)$ or $J(\theta)$ vs n iter

In [17]:
#For the two instances of housing data
ifig=4;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
theta1 = [x[0] for x in theta_house]
theta2 = [x[1] for x in theta_house]
plt.plot(J_house, theta1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#For the two instances of housing data
ifig=5;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
plt.plot(theta1, theta2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
ifig=5;plt.close(ifig);plt.figure(ifig,figsize=(7,6), dpi=120)
niter = range(len(theta1))
plt.plot(niter,theta1)
plt.plot(niter,theta2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …